In [216]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [217]:
# request for the html response using the url for wikipedia
# to scrape the page to see if we get value 200 in return, confirming acceptable to utilize page info

url="https://en.wikipedia.org/wiki/List_of_best-selling_singles_in_the_United_States"
response = requests.get(url)
table_class="wikitable sortable plainrowheaders jquery-tablesorter"
response=requests.get(url)
print(response.status_code)

200


In [218]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
singlesTable=soup.find_all('table',{'class':"wikitable sortable plainrowheaders"})
# singlesTable

In [219]:
# grab the tags with table titles using beautiful soup
title_list = soup.find_all('span',{'class':"mw-headline"})
title_list

[<span class="mw-headline" id="Physical_singles">Physical singles</span>,
 <span class="mw-headline" id="Digital_singles">Digital singles</span>,
 <span class="mw-headline" id="11×_Platinum_or_higher">11× Platinum or higher</span>,
 <span class="mw-headline" id="Diamond_certification">Diamond certification</span>,
 <span class="mw-headline" id="9×_Platinum">9× Platinum</span>,
 <span class="mw-headline" id="8×_Platinum">8× Platinum</span>,
 <span class="mw-headline" id="7×_Platinum">7× Platinum</span>,
 <span class="mw-headline" id="6×_Platinum_or_more_than_6_million_in_sales">6× Platinum or more than 6 million in sales</span>,
 <span class="mw-headline" id="Achievements">Achievements</span>,
 <span class="mw-headline" id="See_also">See also</span>,
 <span class="mw-headline" id="Notes">Notes</span>,
 <span class="mw-headline" id="References">References</span>,
 <span class="mw-headline" id="External_links">External links</span>]

In [220]:
# create clean list of table titles
# *may not be leveraged
for title in title_list:
  title_clean = title.text
  print(title_clean)

Physical singles
Digital singles
11× Platinum or higher
Diamond certification
9× Platinum
8× Platinum
7× Platinum
6× Platinum or more than 6 million in sales
Achievements
See also
Notes
References
External links


In [221]:
len(singlesTable)

7

In [222]:
# create a df from the album table
# loop through, using range to limit observations to desired portions of url
# use pd.concat to append df row to df
df=pd.read_html(str(singlesTable))

# create empty df to allow population in loop
best_selling_singles_df = pd.DataFrame()

for i in range(len(singlesTable)):
  if i == 0:
    df[i].rename({'No. of times Platinum[3]': 'times_platinum','Number of times Platinum[21]': 'times_platinum','Song': 'song', 'Release year': 'released', 'Artist(s)': 'artist', "Claimed sales(million)[A]": "actual_sales"}, axis=1, inplace=True)
    best_selling_singles_df = df[i]
  else:
    df[i].rename({'No. of times Platinum[3]': 'times_platinum','Number of times Platinum[21]': 'times_platinum','Song': 'song','Releaseyear': 'released', 'Artist(s)': 'artist', "Actual sales(million)": "actual_sales", "Actual sales": "actual_sales"}, axis=1, inplace=True)
    best_selling_singles_df = pd.concat([best_selling_singles_df, df[i]])
    best_selling_singles_df["released"] = best_selling_singles_df["released"].astype(int)
    best_selling_singles_df["song"] = best_selling_singles_df["song"].str.strip('"') 
    best_selling_singles_df["actual_sales"] = best_selling_singles_df["actual_sales"].str.replace(r"(\s*\[.*?\]\s*)", " ").str.strip()
    
print(best_selling_singles_df.head())
print(best_selling_singles_df.tail())
print(best_selling_singles_df.columns)

                              song          artist  released actual_sales  \
0                  White Christmas     Bing Crosby      1941           25   
1          Candle in the Wind 1997      Elton John      1997         8.84   
2                 We Are the World  USA for Africa      1985            8   
3  Rudolph, the Red-Nosed Reindeer      Gene Autry      1949            7   
4                       Paper Doll  Mills Brothers      1943            6   

  times_platinum  
0             4×  
1            11×  
2             4×  
3           None  
4           None  
                                          song             artist  released  \
50                                   Fuck Love       XXXTentacion      2018   
51                                  Better Now        Post Malone      2018   
52                                        Ride  Twenty One Pilots      2015   
53                              No Role Modelz            J. Cole      2015   
54  I Wanna Dance with Someb

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.


In [223]:
# iterate through columns using map function to change values to lower case
best_selling_singles_df.columns = map(str.lower, best_selling_singles_df.columns)
best_selling_singles_df.columns

Index(['song', 'artist', 'released', 'actual_sales', 'times_platinum'], dtype='object')

In [224]:
print(f"DF shape: {best_selling_singles_df.shape}")
print(f"DF columns: {best_selling_singles_df.info()}")

DF shape: (217, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 54
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   song            217 non-null    object
 1   artist          217 non-null    object
 2   released        217 non-null    int64 
 3   actual_sales    217 non-null    object
 4   times_platinum  217 non-null    object
dtypes: int64(1), object(4)
memory usage: 10.2+ KB
DF columns: None


In [225]:
# push to a csv
best_selling_singles_df.to_csv('bestsellingsingles.csv', encoding='utf-8', index=False)